In [127]:
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from time import time

In [2]:
headers = {"User-Agent" : 
           "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}
idx = 'https://baike.baidu.com'

In [145]:
# url of all lines
path = idx + '/item/北京地铁/408485'
r = requests.get(path, 
                 allow_redirects=False,headers=headers,verify=False)
r.encoding='utf-8'
line_soup = BeautifulSoup(r.text)
lines = line_soup.find_all(href = re.compile("item"), text=re.compile("北京地铁"))
line_dict = {}
for line in lines:
    line_dict[line.string] = line['href']

In [155]:
#url for stations of each line
station_dict = {}
for line_name in line_dict:
    path = idx + line_dict[line_name]
    print(line_name, path)
    r = requests.get(path, 
                 allow_redirects=False,headers=headers,verify=False)
    r.encoding='utf-8'
    station_soup = BeautifulSoup(r.text)
    stations = station_soup.find_all(href = re.compile("item"), text=re.compile("站"))
    station_dict[line_name] = [[station.text, station['href']] for station in stations]

北京地铁1号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%811%E5%8F%B7%E7%BA%BF
北京地铁13号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8113%E5%8F%B7%E7%BA%BF
北京地铁八通线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E5%85%AB%E9%80%9A%E7%BA%BF
北京地铁5号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%815%E5%8F%B7%E7%BA%BF
北京地铁8号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%818%E5%8F%B7%E7%BA%BF
北京地铁10号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8110%E5%8F%B7%E7%BA%BF
北京地铁机场线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E6%9C%BA%E5%9C%BA%E7%BA%BF
北京地铁4号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%814%E5%8F%B7%E7%BA%BF
北京地铁15号线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8115%E5%8F%B7%E7%BA%BF
北京地铁昌平线 https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E6%98%8C%E5%B9%B3%E7%BA%BF
北京地铁大兴线 https://baike.baidu.com/item/%E5

In [128]:
def get_response(url):
    r = requests.get(url,headers=headers,verify=False)
    r.encoding='utf-8'
    return r
def get_connections(url):
    r = get_response(url)
    if r.status_code is not 200:
        for i in range(3):#访问失败时重试3次
            time.sleep(0.5)
            r = get_response(url)
            if r.status_code == 200:
                break
        if r.status_code is not 200: 
            print(r.status_code, url)
            return '无法访问：' + url, []
    soup = BeautifulSoup(r.text)
    print(r.status_code, soup.h1, url)
    current_station = soup.h1.text
    connections = []
    for result in soup.find_all(href = re.compile("item"), text = re.compile("站")):
        if re.search(r"(下一站)|(下站)", str(result.parent)):
            connections.append([result.text, result['href']])
    return current_station, connections

In [129]:
station_connections = {}
start_url = "/item/%E5%9B%9B%E6%83%A0%E4%B8%9C%E7%AB%99"
visited = [start_url]
seen = set()
while visited:
    time.sleep(0.5)
    current_url = idx + visited.pop()
    if current_url in seen: continue
    current_station, connections = get_connections(current_url)
    station_connections[current_station] = connections
    for connection in connections:
        if connection[1] in seen: continue
        visited = [connection[1]] + visited
    seen.add(current_url)

200 <h1>四惠东站</h1> https://baike.baidu.com/item/%E5%9B%9B%E6%83%A0%E4%B8%9C%E7%AB%99
200 <h1>四惠站</h1> https://baike.baidu.com/item/%E5%9B%9B%E6%83%A0%E7%AB%99
200 <h1>四惠站</h1> https://baike.baidu.com/item/%E5%9B%9B%E6%83%A0%E7%AB%99/2686893
200 <h1>高碑店站</h1> https://baike.baidu.com/item/%E9%AB%98%E7%A2%91%E5%BA%97%E7%AB%99/15410420
200 <h1>大望路站</h1> https://baike.baidu.com/item/%E5%A4%A7%E6%9C%9B%E8%B7%AF%E7%AB%99
200 <h1>四惠东站</h1> https://baike.baidu.com/item/%E5%9B%9B%E6%83%A0%E4%B8%9C%E7%AB%99/1229653
200 <h1>传媒大学站</h1> https://baike.baidu.com/item/%E4%BC%A0%E5%AA%92%E5%A4%A7%E5%AD%A6%E7%AB%99
200 <h1>国贸站</h1> https://baike.baidu.com/item/%E5%9B%BD%E8%B4%B8%E7%AB%99/4341806
200 <h1>九龙山站</h1> https://baike.baidu.com/item/%E4%B9%9D%E9%BE%99%E5%B1%B1%E7%AB%99/15085203
200 <h1>红庙站</h1> https://baike.baidu.com/item/%E7%BA%A2%E5%BA%99%E7%AB%99
200 <h1>双桥站</h1> https://baike.baidu.com/item/%E5%8F%8C%E6%A1%A5%E7%AB%99/16513778
200 <h1>永安里站</h1> https://baike.baidu.com/item/%E6%B0%B8%E5%AE%89

200 <h1>草房站</h1> https://baike.baidu.com/item/%E8%8D%89%E6%88%BF%E7%AB%99
200 <h1>望京南站</h1> https://baike.baidu.com/item/%E6%9C%9B%E4%BA%AC%E5%8D%97%E7%AB%99
200 <h1>芳园里站</h1> https://baike.baidu.com/item/%E8%8A%B3%E5%9B%AD%E9%87%8C%E7%AB%99
200 <h1>酒仙桥站</h1> https://baike.baidu.com/item/%E9%85%92%E4%BB%99%E6%A1%A5%E7%AB%99
200 <h1>长椿街站</h1> https://baike.baidu.com/item/%E9%95%BF%E6%A4%BF%E8%A1%97%E7%AB%99
200 <h1>菜市口站</h1> https://baike.baidu.com/item/%E8%8F%9C%E5%B8%82%E5%8F%A3%E7%AB%99
200 <h1>永定门外站</h1> https://baike.baidu.com/item/%E6%B0%B8%E5%AE%9A%E9%97%A8%E5%A4%96%E7%AB%99
200 <h1>和平西桥站</h1> https://baike.baidu.com/item/%E5%92%8C%E5%B9%B3%E8%A5%BF%E6%A1%A5%E7%AB%99
200 <h1>芍药居站</h1> https://baike.baidu.com/item/%E8%8A%8D%E8%8D%AF%E5%B1%85%E7%AB%99
200 <h1>和平西桥站</h1> https://baike.baidu.com/item/%E5%92%8C%E5%B9%B3%E8%A5%BF%E6%A1%A5%E7%AB%99/9255996
200 <h1>香河园站</h1> https://baike.baidu.com/item/%E9%A6%99%E6%B2%B3%E5%9B%AD%E7%AB%99
200 <h1>望京西站</h1> https://baike.baidu.com/item/%

200 <h1>西铁营站</h1> https://baike.baidu.com/item/%E8%A5%BF%E9%93%81%E8%90%A5%E7%AB%99
200 <h1>草桥站</h1> https://baike.baidu.com/item/%E8%8D%89%E6%A1%A5%E7%AB%99/15415821
200 <h1>牛街站</h1> https://baike.baidu.com/item/%E7%89%9B%E8%A1%97%E7%AB%99
200 <h1>北京西站</h1> https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E8%A5%BF%E7%AB%99/17646705
200 <h1>火箭万源站</h1> https://baike.baidu.com/item/%E7%81%AB%E7%AE%AD%E4%B8%87%E6%BA%90%E7%AB%99/22639889
200 <h1>角门东站</h1> https://baike.baidu.com/item/%E8%A7%92%E9%97%A8%E4%B8%9C%E7%AB%99
200 <h1>石榴庄站</h1> https://baike.baidu.com/item/%E7%9F%B3%E6%A6%B4%E5%BA%84%E7%AB%99
200 <h1>大红门南站</h1> https://baike.baidu.com/item/%E5%A4%A7%E7%BA%A2%E9%97%A8%E5%8D%97%E7%AB%99/22639820
200 <h1>双岛式站台</h1> https://baike.baidu.com/item/%E5%8F%8C%E5%B2%9B%E5%BC%8F%E7%AB%99%E5%8F%B0
200 <h1>回龙观站</h1> https://baike.baidu.com/item/%E5%9B%9E%E9%BE%99%E8%A7%82%E7%AB%99
200 <h1>育新站</h1> https://baike.baidu.com/item/%E8%82%B2%E6%96%B0%E7%AB%99
200 <h1>回龙观东大街站</h1> https://baike.baidu

200 <h1>人民大学站</h1> https://baike.baidu.com/item/%E4%BA%BA%E6%B0%91%E5%A4%A7%E5%AD%A6%E7%AB%99
200 <h1>高米店南站</h1> https://baike.baidu.com/item/%E9%AB%98%E7%B1%B3%E5%BA%97%E5%8D%97%E7%AB%99
200 <h1>首经贸站</h1> https://baike.baidu.com/item/%E9%A6%96%E7%BB%8F%E8%B4%B8%E7%AB%99
200 <h1>丰台南路站</h1> https://baike.baidu.com/item/%E4%B8%B0%E5%8F%B0%E5%8D%97%E8%B7%AF%E7%AB%99
200 <h1>八宝山站</h1> https://baike.baidu.com/item/%E5%85%AB%E5%AE%9D%E5%B1%B1%E7%AB%99
200 <h1>田村站</h1> https://baike.baidu.com/item/%E7%94%B0%E6%9D%91%E7%AB%99/18549433
200 <h1>四季青站</h1> https://baike.baidu.com/item/%E5%9B%9B%E5%AD%A3%E9%9D%92%E7%AB%99
200 <h1>长春桥站</h1> https://baike.baidu.com/item/%E9%95%BF%E6%98%A5%E6%A1%A5%E7%AB%99
200 <h1>大钟寺站</h1> https://baike.baidu.com/item/%E5%A4%A7%E9%92%9F%E5%AF%BA%E7%AB%99
200 <h1>沙河站</h1> https://baike.baidu.com/item/%E6%B2%99%E6%B2%B3%E7%AB%99/15092700
200 <h1>石门站</h1> https://baike.baidu.com/item/%E7%9F%B3%E9%97%A8%E7%AB%99/15829494
200 <h1>巴沟站</h1> https://baike.baidu.com/item/%E5

In [157]:
all_station = set()
for line_name in station_dict:
    stations = station_dict[line_name]
    for station in stations:
        all_station.add(station[0])

In [169]:
all_visited_station = set(station_connections.keys())

In [173]:
len(all_station^all_visited_station)

140

In [ ]:
def bfs(graph, start):
    visited = [start]
    seen = set()
    while visited:
        froninter = visited.pop() #
        if froninter in seen: continue
        for successor in graph[froninter]:
            if successor in seen: continue
            print(successor)
            visited = [successor] + visited
        seen.add(froninter)
    return seen